In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import cassio

ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_ID = os.getenv("ASTRA_DB_ID")

cassio.init(
    token = ASTRA_DB_APPLICATION_TOKEN,
    database_id = ASTRA_DB_ID
)

In [ ]:
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

## Load and Split Documents

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain.vectorstores.cassandra import Cassandra

astra_vector_store = Cassandra(
    embedding = embeddings,
    table_name = "qa_mini_demo",
    session = None,
    keyspace = None
)

In [ ]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

astra_vector_store.add_documents(doc_splits)
print("Inserted %i headlines." % len(doc_splits))
astra_vector_index = VectorStoreIndexWrapper(vectorstore = astra_vector_store)

In [ ]:
retriever = astra_vector_store.as_retriever()

retriever.invoke("What is agent",ConsistencyLevel="LOCAL_ONE")

## Router

In [ ]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

class RouteQuery(BaseModel):
    datadource: Literal['vectorstore', 'wiki_search'] = Field(
        ...,
        description = "Given a user query, chose to search in vectorstore or wiki_search"
    )

In [ ]:
from google.colab import userdata
from langchain_groq import ChatGroq

groq_api_key = userdata.get('groq_api_key')
os.environ["GROQ_API_KEY"] = groq_api_key

In [ ]:
llm = ChatGroq(
    groq_api_key = groq_api_key,
    model_name = "Gemma2-9b-It"
)

structured_llm_router = llm.with_structured_output(RouteQuery)

In [ ]:
# Prompt
system = """You are an expert at routing a user question to a vectorstore or wikipedia.
The vectorstore contains documents related to agents, prompt engineering, and adversarial attacks.
Use the vectorstore for questions on these topics. Otherwise, use wiki-search."""

route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

question_router = route_prompt | structured_llm_router
print(
    question_router.invoke(
        {"question": "who is Sharukh Khan?"}
    )
)
print(question_router.invoke({"question": "What are the types of agent memory?"}))